<a href="https://colab.research.google.com/github/WTeam2024/rag_chatbot/blob/main/chatbot_WTeam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install required packages
!pip install -q peft  accelerate bitsandbytes safetensors sentencepiece streamlit chromadb langchain sentence-transformers gradio pypdf langchain-community
!pip install -U transformers
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate

In [ ]:
# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# import dependencies
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

import os
import gradio as gr
from google.colab import drive

import chromadb
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain import HuggingFacePipeline
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [ ]:
# specify model huggingface mode name
model_name = "galatolo/cerbero-7b"

In [ ]:
# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    return model

In [ ]:
# fucntion for initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

In [ ]:
from google.colab import userdata
userdata.get('HF_TOKEN')

In [ ]:
# !pip install huggingface_hub
# from huggingface_hub import *

# from transformers import AutoModelForCausalLM, AutoTokenizer
# from huggingface_hub import login

# # Login to Hugging Face Hub
# #huggingfacehub_api_token = 'HF_TOKEN'
# #login(token=huggingfacehub_api_token)

# # Caricare il tokenizer e il modello
# repo_id = 'tiiuae/falcon-7b-instruct'
# tokenizer = AutoTokenizer.from_pretrained(repo_id)
# model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b-instruct")

In [ ]:
### PER FAR FUNZIONARE QUESTA PARTE: VAI SU RUNTIME -> CHANGE RUNTIME TYPE -> SELEZIONA T4 GPU!

# load model
model = load_quantized_model(model_name)

# initialize tokenizer
tokenizer = initialize_tokenizer(model_name)

# specify stop token ids
stop_token_ids = [0]

In [ ]:
# mount google drive and specify folder path
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/documents_colab/'

In [ ]:
### SPOSTA I PDFs IN UN'UNICA CARTELLA
import os
import shutil

# Specifica la directory di partenza e la directory di destinazione
source_dir = folder_path
destination_dir = '/content/PDFs'

# Controlla se la directory di destinazione esiste, altrimenti creala
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Funzione per spostare i file PDF
def move_pdfs(source, destination):
    for root, dirs, files in os.walk(source):
        for file in files:
            if file.lower().endswith('.pdf'):
                source_file = os.path.join(root, file)
                destination_file = os.path.join(destination, file)
                shutil.copy(source_file, destination_file)
                print(f'Spostato: {source_file} -> {destination_file}')

# Esegui la funzione
move_pdfs(source_dir, destination_dir)
folder_path = destination_dir

In [ ]:
# load pdf files
loader = PyPDFDirectoryLoader(folder_path)
documents = loader.load()

In [ ]:
# split the documents in small chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200) #Chage the chunk_size and chunk_overlap as needed
all_splits = text_splitter.split_documents(documents)

# Check the first few splits to ensure coherence
for i, split in enumerate(all_splits[:3]):
    print(f"Chunk {i+1}:\n{split}\n{'-'*20}")

In [ ]:
# specify embedding model (using huggingface sentence transformer)
embedding_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
#device = torch.device('cpu')
model_kwargs = {"device": "cuda"}

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)

#embed document chunks
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

# specify the retriever
retriever = vectordb.as_retriever()

In [ ]:
# build huggingface pipeline for using zephyr-7b-alpha
pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=2048,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

# specify the llm
llm = HuggingFacePipeline(pipeline=pipeline)

In [ ]:
# build conversational retrieval chain with memory (rag) using langchain
def create_conversation(query: str, chat_history: list) -> tuple:
    try:

        memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=False
        )
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retriever,
            memory=memory,
            get_chat_history=lambda h: h,
        )

        result = qa_chain({'question': query, 'chat_history': chat_history})
        chat_history.append((query, result['answer']))
        return '', chat_history

    except Exception as e:
        chat_history.append((query, e))
        return '', chat_history

In [ ]:
# build gradio ui
#with gr.Blocks() as demo:

#    gr.Markdown("<h1 align='center'>Personal Chatbot</h1>")
#    chatbot = gr.Chatbot(label='Textmining Project - W_Team - 2023/2024')
#    state = gr.State([])  # Cronologia della chat

#    with gr.Row():
#        with gr.Column(scale=9):  # La casella di testo occupa la maggior parte della larghezza
#            msg = gr.Textbox(show_label=False, placeholder="Enter question and press enter",lines=2)
#        with gr.Column(scale=1):  # Il pulsante occupa una parte minore della larghezza
#            submit_btn = gr.Button("Send",size="sm")
#            clear_btn = gr.Button("Clear", size="sm")

    #clear = gr.ClearButton([msg, chatbot])
#    clear_btn.click(fn=lambda: ("", []), inputs=[], outputs=[msg, chatbot])
#    submit_btn.click(fn=create_conversation, inputs=[msg, chatbot], outputs=[msg, chatbot],concurrency_limit=1)
    #submit_btn.click(fn=create_conversation, inputs=msg, outputs=gr.ChatInterface, concurrency_limit=1)
#    msg.submit(create_conversation, [msg, chatbot], [msg, chatbot],concurrency_limit=1)

#demo.launch()

In [ ]:
with gr.Blocks() as demo:

    gr.Markdown("<h1 align='center'>Personal Chatbot</h1>")
    chatbot = gr.Chatbot()
    #msg = gr.Textbox(label="Your message:")
    state = gr.State([])  # Cronologia della chat

    with gr.Row():
        msg = gr.Textbox(show_label=False, placeholder="Enter question and press enter")
    clear = gr.ClearButton([msg, chatbot])
    gr.themes.Soft()

    msg.submit(create_conversation, [msg, chatbot], [msg, chatbot])

demo.launch(debug=True,share=True)